In [1]:
import Auto3D
from Auto3D.auto3D import options, main

In [2]:
#Always ensure that you have the latest version
print(Auto3D.__version__)

0.1.2


# Generate low-energy conformers with Auto3D

In [3]:
if __name__ == "__main__":
    path = "/Users/liu5/Documents/Auto3D_pkg/example/files/smiles.smi"
    args = options(path, k=1, use_gpu=False)   #args specify the parameters for Auto3D 
    out = main(args)            #main acceps the parameters and run Auto3D
    print(out)

Checking input file...
	There are 4 SMILES in the input file /Users/liu5/Documents/Auto3D_pkg/example/files/smiles.smi. 
	All SMILES and IDs are valid.
Suggestions for choosing isomer_engine and optimizing_engine: 
	Isomer engine options: RDKit and Omega.
	Optimizing engine options: ANI2x, ANI_2xt and AIMNET.
There are 4 SMILES, available memory is 16.0 GB.
The task will be divided into 1 jobs.
Job1, number of inputs: 4


Isomer generation for job1
Enumerating cis/tran isomers for unspecified double bonds...
Enumerating R/S isomers for unspecified atomic centers...
Removing enantiomers...
Enumerating conformers/rotamers, removing duplicates...


Optimizing on job1
Preparing for parallel optimizing... (Max optimization steps: 10000)
Total 3D conformers: 54


 10%|█         | 1000/10000 [03:44<15:29,  9.68it/s]

Total 3D structures: 54  Converged: 38   Active: 16


 20%|██        | 2002/10000 [04:24<02:06, 63.06it/s]

Total 3D structures: 54  Converged: 53   Active: 1


 30%|██▉       | 2998/10000 [04:40<01:53, 61.80it/s]

Total 3D structures: 54  Converged: 53   Active: 1


 37%|███▋      | 3680/10000 [04:50<08:19, 12.66it/s]


All structures converged at step 3681:   Total 3D structures: 54  Converged: 54   Active: 0
Beggin to slelect structures that satisfy the requirements...
Energy unit: Hartree if implicit.
Program running time: 297.10353779792786 seconds
/Users/liu5/Documents/Auto3D_pkg/example/files/20220502-113701_smiles/smiles_out.sdf


In [4]:
help(options)

Help on function options in module Auto3D.auto3D:

options(path, k=False, window=False, verbose=False, job_name='', enumerate_tautomer=False, tauto_engine='rdkit', isomer_engine='rdkit', cis_trans=True, mode_oe='classic', mpi_np=4, max_confs=1000, use_gpu=True, gpu_idx=0, capacity=42, optimizing_engine='AIMNET', opt_steps=10000, convergence_threshold=0.003, threshold=0.3)
    Arguments for Auto3D main program
    path: A input.smi containing SMILES and IDs. Examples are listed in the example/files folder
    k: Outputs the top-k structures for each SMILES.
    window: Outputs the structures whose energies are within x (kcal/mol) from the lowest energy conformer
    verbose: When True, save all meta data while running.
    job_name: A folder name to save all meta data.
    
    enumerate_tautomer: When True, enumerate tautomers for the input
    taut_engine: Programs to enumerate tautomers, either 'rdkit' or 'oechem'
    isomer_engine: The program for generating 3D isomers for each SMIL

In [5]:
help(main)

Help on function main in module Auto3D.auto3D:

main(args: dict)
    Take the arguments from options and run Auto3D



# Calculate thermodynamic properties with the 3D structures

In [8]:
from Auto3D.ASE.thermo import calc_thermo

In [10]:
#need to define a function that gets the unique ID and temperature for molecules in the input path.
#The mol object is pybel Molecule object
def get_mol_idx_t(mol):
    id = mol.data["ID"]
    t = 298
    return (id, t)


out_thermo = calc_thermo(out, "AIMNET", get_mol_idx_t)
print(out_thermo)

      Step     Time          Energy         fmax
BFGS:    0 11:48:22   -13060.592595        0.0041
Enthalpy components at T = 298.00 K:
E_pot             -13060.593 eV
E_ZPE                  5.084 eV
Cv_trans (0->T)        0.039 eV
Cv_rot (0->T)          0.039 eV
Cv_vib (0->T)          0.195 eV
(C_v -> C_p)           0.026 eV
-------------------------------
H                 -13055.211 eV
Entropy components at T = 298.00 K and P = 101325.0 Pa:
                           S               T*S
S_trans (1 bar)    0.0017751 eV/K        0.529 eV
S_rot              0.0013111 eV/K        0.391 eV
S_elec             0.0000000 eV/K        0.000 eV
S_vib              0.0012050 eV/K        0.359 eV
S (1 bar -> P)    -0.0000011 eV/K       -0.000 eV
-------------------------------------------------
S                  0.0042900 eV/K        1.278 eV
Enthalpy components at T = 298.00 K:
E_pot             -13060.593 eV
E_ZPE                  5.084 eV
Cv_trans (0->T)        0.039 eV
Cv_rot (0->T)         

In [9]:
help(calc_thermo)

Help on function calc_thermo in module Auto3D.ASE.thermo:

calc_thermo(path: str, model_name: str, get_mol_idx_t: <built-in function callable>, gpu_idx=0, opt_tol=0.005, opt_steps=10000)
    ASE interface for calculation thermo properties using ANI2xt and AIMNET
    path: Input sdf file
    model_name: ANI2x, ANI2xt or AIMNET
    get_mol_idx_t: a functioin that returns (idx, T) from a pybel mol object
    gpu_idx: GPU cuda index
    opt_tol: Convergence_threshold for geometry optimization
    opt_steps: Maximum geometry optimizaiton steps

